# Training with HDF5 Data

In order to allow larger sets of data to be processed with limited system memory we take advantage of HDF5 and the [h5py package](https://www.h5py.org/).

This allows us to incrementally save our pre-processed and randomised data to disk, and then incrementally load the data using iterable tensors for training.

In contrast to the main example notebook `AstroNoise2Noise.ipynb`, in this example we train with two sets of subs from different targets resulting in substantially more data.

*Enhancements have been provided by Maria Pavlou as code changes to the base CDBDeep repo.*

In [12]:
# Allow reloading of CSBDeep modules following any code changes
%reload_ext autoreload
%autoreload 2

# A couple required imports
import numpy as np
from tifffile import imread
from pathlib import Path

# Pre-processing Training Data and Save

We have two sets of sub frames in the `data/astro` sub-folders `NGC6888` and `NGC7000`, acquired with the same optical train and equipment.

All images have been aligned and calibrated and saved in the tiff format.

The example data can be downloaded [here](https://1drv.ms/u/s!AvWEkn9Anb_Nq9Aw52Xs3LuYEcq_rg?e=EexXxL)

Place the train images in sub-folders under the `data/astro` folder.

In [21]:
# Setup Data Parameters:
# Root Data path
basepath=Path('data/astro')
# Train Data path/s
source_dirs=['NGC6888','NGC7000']
# Image file pattern. Note: only formats supported by imread currently
pattern='*.tiff'
# Image patch size
patchsize=64
# Training data output savefile path & name
training_data_name='NGC7000_NGC6888_p64_nPERC'
training_data_filename=training_data_name + '.hdf5'
save_file=basepath/training_data_filename

In [22]:
# Make an estimate of the number of non-overlapping patches for the images we have, sampling from the first we find.
first_image_file = list((basepath/source_dirs[0]).glob(pattern))[0]
sampleimage = imread(first_image_file)
n_patches_per_image=np.int(sampleimage.shape[0]/patchsize)*np.int(sampleimage.shape[1]/patchsize)

# Save pre-processed data to HDF5

No we use the `create_patches_hdf5` helper to create the sampled patches from the sub frame images, and store these directly to disk as an HDF5 file.

**Note:**

* Saving to disk will take longer, and significantly longer depending on your system configuration
* Randomised indexing cannot be used when writing to HDF5 Datasets. A secondary read/write process is used to achieve a compromised shuffle on the raw data. This can take some time, but leads to improved performance during training. A good tradeoff.

In [23]:
from csbdeep.data import RawData, create_patches, create_patches_hdf5, norm_percentiles, norm_reinhard
from csbdeep.data import NoPreProcessor, ReinhardPreProcessor

# Load image pairs for Noise2Noise processing, each image paired against every other at most once.
raw_data = RawData.from_folder_n2n(basepath, source_dirs=source_dirs, axes='YXC', pattern=pattern, preprocessor=NoPreProcessor(), imageloader=None)

# Create patch data from image pairs with parameters,
# normalization set as norm_percentiles() by default, optionally set to None, norm_reinhard() or other custom
create_patches_hdf5(
    raw_data, 
    patch_size=(patchsize,patchsize,3),
    normalization=norm_percentiles(),
    n_patches_per_image=n_patches_per_image,
    save_file=save_file,
    patch_filter=None,
    overlap=False,
    shuffle=True)

  380 raw images x    1 transformations   =   380 images
  380 images     x  884 patches per image = 335920 patches in total
Input data:
data\astro:, sources='['NGC6888', 'NGC7000']', axes='YXC', pattern='*.tiff'
Transformations:
1 x Identity
Patch size:
64 x 64 x 3


d:\pydeep\astro-csbdeep\csbdeep\data\generate_hdf5.py:111: RuntimeWarning: overflow encountered in long_scalars
  n_required_memory_bytes = 2 * n_patches*np.prod(patch_size) * 4
100%|██████████| 380/380 [09:57<00:00,  1.57s/it]


Saved as hdf5 data to data\astro\NGC7000_NGC6888_p64_nPERC.hdf5.


# Load Training Data
Here we load the training data from a save file created in earlier steps.
As data is loaded we can also split the data into training `X,Y` and validation `X_val,Y_val` sets.

In [27]:
from csbdeep.data.generate_hdf5 import HDF5Data

# Train/Validation split %
validation_split=0.1
# Select 1st channel initially
channel_slice = slice(0,1)
# Since we have shuffles the raw data we disable this for performance
hdf5_shuffled_read = False

# Load saved training and validation from HDF5 data with iterable wrapper object and channel selection
train_data, val_data = HDF5Data.from_hdf5(save_file, validation_split=validation_split, channels=channel_slice, shuffled_read=hdf5_shuffled_read)

print('Train Data Shape =', train_data.shape)
print('Val Data Shape =', val_data.shape)

Train Data Shape = (302328, 64, 64, 3)
Val Data Shape = (33592, 64, 64, 3)


# Configure and Train the Learning Model

Here we configure the training model parameters. 
Training will be done for each color channel separately and saved with individual names based on:

* `training_data_name`
* `model_base_name`
* `channel_name[i]`

## Training debug tools

You can monitor the progress during training with [TensorBoard](https://www.tensorflow.org/programmers_guide/summaries_and_tensorboard) by starting it from the current working directory:

    $ tensorboard --logdir=.

Then connect to [http://localhost:6006/](http://localhost:6006/) with your browser.

In [29]:
# Importing the deep learning framework
from csbdeep.models import Config, CARE, HDF5CARE

# Since we are training each channel separately: n_channel_in, n_channel_out = 1, 1
# Probabilistic training will be used as this yields better results
# Reccomended that train_steps_per_epoch set at 200-400, and train_epochs at 100-200 for good results.
config = Config('SYXC', n_channel_in=1, n_channel_out=1, unet_kern_size=3, probabilistic=True, train_steps_per_epoch=200, train_epochs=50)

# Give a name for the model
model_base_name = '_PROB_spe400_e100_hdf5_main'

model_name = training_data_name + model_base_name
skipindex = [ ]
channel_names=['R', 'G', 'B']
for i, channel in enumerate(channel_names):
    if i in skipindex:
        continue
    
    # update the HDF5 iterable channel slice selection
    train_data.set_channel(slice(i,i+1))
    val_data.set_channel(slice(i,i+1))

    full_model_name = model_name + '_' + channel
    
    # Create the Learning Model from the CARE framework with configuration
    model = HDF5CARE(config, name=full_model_name, basedir='models')
        
    # Train the model and capture history
    history = model.train(
        XY_data=train_data, validation_data=val_data[:])
    
    # Save the model
    model.export_TF()

d:\pydeep\astro-csbdeep\csbdeep\models\care_hdf5.py:46: UserWarning: small number of validation images (only 0.0% of all images)
  warnings.warn("small number of validation images (only %.1f%% of all images)" % (100*frac_val))
d:\pydeep\astro-csbdeep\.venv\lib\site-packages\keras\optimizer_v2\optimizer_v2.py:355: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(


Epoch 1/50
Cause: could not parse the source code of <function _mean_or_not.<locals>.<lambda> at 0x000002040871E160>: found multiple definitions with identical signatures at the location. This error may be avoided by defining each lambda on a single line and with unique argument names.
Match 0:
(lambda x: K.mean(x, axis=(- 1)))

Match 1:
(lambda x: x)

To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Cause: could not parse the source code of <function _mean_or_not.<locals>.<lambda> at 0x000002040871E160>: found multiple definitions with identical signatures at the location. This error may be avoided by defining each lambda on a single line and with unique argument names.
Match 0:
(lambda x: K.mean(x, axis=(- 1)))

Match 1:
(lambda x: x)

To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Cause: could not parse the source code of <function _mean_or_not.<locals>.<lambda> at 0x000002040871E310>: found 

d:\pydeep\astro-csbdeep\csbdeep\utils\tf.py:177: UserWarning: 
***IMPORTANT NOTE***

You are using 'tensorflow' 2.x, hence it is likely that the exported model *will not work*
in associated ImageJ/Fiji plugins (e.g. CSBDeep and StarDist).

If you indeed have problems loading the exported model in Fiji, the current workaround is
to load the trained model in a Python environment with installed 'tensorflow' 1.x and then
export it again. If you need help with this, please read:

https://gist.github.com/uschmidt83/4b747862fe307044c722d6d1009f6183

  warnings.warn(\



Model exported in TensorFlow's SavedModel format:
D:\pydeep\astro-csbdeep\models\NGC7000_NGC6888_p64_nPERC_PROB_spe400_e100_hdf5_main_R\TF_SavedModel.zip
Epoch 1/50
Cause: could not parse the source code of <function _mean_or_not.<locals>.<lambda> at 0x000002044BEB43A0>: found multiple definitions with identical signatures at the location. This error may be avoided by defining each lambda on a single line and with unique argument names.
Match 0:
(lambda x: K.mean(x, axis=(- 1)))

Match 1:
(lambda x: x)

To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Cause: could not parse the source code of <function _mean_or_not.<locals>.<lambda> at 0x000002044BEB43A0>: found multiple definitions with identical signatures at the location. This error may be avoided by defining each lambda on a single line and with unique argument names.
Match 0:
(lambda x: K.mean(x, axis=(- 1)))

Match 1:
(lambda x: x)

To silence this warning, decorate the function with 

# Load and De-noise an Image 

Here we load an example image to de-noise using the trained set of RGB models.

The image to be de-noised must be normalized in the same way as the training data.
Then for each channel the model is used to predict the de-noised output and these are then saved as a single RGB image.

The example test image used can be downloaded [here](https://1drv.ms/u/s!AvWEkn9Anb_Nq9Aw52Xs3LuYEcq_rg?e=EexXxL).

Place the test image in the `data/astro/test` folder.

In [30]:
# Specify the test file
test_file_name='CrescentNebula-NoSt-Deep.tiff'
#test_file_name='FireDog-Comb33-NoSt.tiff'

testfilepath=basepath/'test'/test_file_name
x = imread(testfilepath)
testaxes = 'YX'

print('Test Image size =', x.shape)
print('Test Image axes =', testaxes)

Test Image size = (2771, 4048, 3)
Test Image axes = YX


In [31]:
from csbdeep.data import PercentileNormalizer, PadAndCropResizer, ReinhardNormalizer, NoNormalizer

channel_names=['R', 'G', 'B']
output_denoised = []
for i, channel in enumerate(channel_names):
    full_model_name = model_name + '_' + channel
    
    # Load the model for the specific channel
    print("Loading model:", full_model_name)
    model = HDF5CARE(config=None, name=full_model_name, basedir='models')

    # Predict/de-noise the image channel with the corresponding trained model
    # Default PercentileNormalizer is used to match the normalization used to train the model
    output_denoised.append(
        model.predict(x[:,:,i],testaxes, normalizer=PercentileNormalizer(), resizer=PadAndCropResizer())
    )

Loading model: NGC7000_NGC6888_p64_nPERC_PROB_spe400_e100_hdf5_main_R
Loading network weights from 'weights_best.h5'.
Out of memory, retrying with n_tiles = (1, 2)
Out of memory, retrying with n_tiles = (2, 2)


100%|██████████| 4/4 [00:00<00:00,  9.01it/s]


Loading model: NGC7000_NGC6888_p64_nPERC_PROB_spe400_e100_hdf5_main_G
Loading network weights from 'weights_best.h5'.
Out of memory, retrying with n_tiles = (1, 2)


100%|██████████| 2/2 [00:00<00:00,  8.26it/s]


Loading model: NGC7000_NGC6888_p64_nPERC_PROB_spe400_e100_hdf5_main_B
Loading network weights from 'weights_best.h5'.
Out of memory, retrying with n_tiles = (1, 2)


100%|██████████| 2/2 [00:00<00:00,  8.13it/s]


In [32]:
# Load astropy library for saving the de-noised image in fits format
from astropy.io import fits

output_file_name = model_name + '_RGB_' + Path(test_file_name).stem + '.fits'
output_file_path = basepath/'test'/output_file_name
hdu = fits.PrimaryHDU(output_denoised)
hdul = fits.HDUList([hdu])
hdul.info()
hdul.writeto(output_file_path)
print("Output file saved:", output_file_path)

Filename: (No file associated with this HDUList)
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU       7   (4048, 2771, 3)   float32   
Output file saved: data\astro\test\NGC7000_NGC6888_p64_nPERC_PROB_spe400_e100_hdf5_main_RGB_CrescentNebula-NoSt-Deep.fits
